### **Importing Necessary Libraries**

In [1]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
import uuid

### **Creating Floder Structure**

In [3]:
# Setup Paths
pos_path = os.path.join('Data', 'Positive')
neg_path = os.path.join('Data', 'Negative')
anc_path = os.path.join('Data', 'Anchor')

In [10]:
# Making the Directories
os.makedirs(pos_path)
os.makedirs(neg_path)
os.makedirs(anc_path)

### **Data Collection**

- #### **Negative Class**

In [8]:
# Moving lfw images to negative directory
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        source_path = os.path.join('lfw', directory, file)
        new_path = os.path.join(neg_path, file)
        os.replace(source_path, new_path)

- #### **Possitive and Anchor Class**

In [43]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()

    # Cropping frames
    frame = frame[90 : 90 + 250, 240 : 240 + 250, :]

    # Collecting anchor class
    if cv2.waitKey(1) & 0XFF == ord('a'):
        img_name = os.path.join(anc_path, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)


    # Collecting positive class
    if cv2.waitKey(1) & 0XFF == ord('p'):
        img_name = os.path.join(pos_path, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)


    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()

### **Loading and Preprocessing**

- #### **Loading Image Direstories**

In [5]:
anchor = tf.data.Dataset.list_files(anc_path + '\*.jpg').take(300)
positive = tf.data.Dataset.list_files(pos_path + '\*.jpg').take(300)
negative = tf.data.Dataset.list_files(neg_path + '\*.jpg').take(300)

- #### **Preprocessing**

In [9]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Resizing the image 
    img = tf.image.resize(img, (105, 105))
    # Scale the image
    img = img / 255.0

    # Return image
    return img

In [11]:
def preprocess_dataset(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

- #### **Creating Labeled Dataset**

In [10]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

- #### **Train Test Split**

In [12]:
# Building dataloader pipeline
data = data.map(preprocess_dataset)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [13]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [19]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)